In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
from pandas import DataFrame
import pandas as pd
from notebook_utils import NotebookUtilities
import numpy as np

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829277, 113)



# Identify any Anomalous Files

In [4]:

anomalous_files_set = set()
anomalous_files_str = ''

In [5]:

# If all the patients in a file are all named Mike, that is a training simulation
files_list = []

# Group the DataFrame by 'file_name'
for file_name, file_name_df in frvrs_logs_df.groupby('file_name'):

    # Get a list of the unique patient IDs in the file
    patient_ids = file_name_df.patient_id.unique().tolist()

    # Check if all the patient IDs in the file contain the string "mike" in lowercase
    if all(map(lambda x: 'mike' in str(x).lower(), patient_ids)):

        # Add the file name to the files_list list
        files_list.append(file_name)

# If there are files with all patients named "Mike," print and update the results
if files_list:

    # Create a string with the list of files having "Mike" patients
    print_str = f'\n\nThese files have patients that are all named "Mike":\n\t{nu.conjunctify_nouns(files_list)}'

    # Add carriage returns for better readability
    print_str = print_str.replace(' and ', ', and\n\t')
    
    # Print the results
    print(print_str)

    # Update the anomalous files string
    anomalous_files_str += print_str

    # Update the anomalous files set
    anomalous_files_set.update(files_list)

In [6]:

# Initialize an empty list to store rows
rows_list = []

# Loop through sessions and scenes
for (session_uuid, scene_index), scene_df in fu.get_session_groupby(extra_column='scene_index'):

    # Create a dictionary to store the results for the current group
    row_dict = {}

    # Get the logger version for the current group
    logger_version = fu.get_logger_version(scene_df)

    # Add the logger version, session UUID, and scene to the dictionary
    for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)
    row_dict['logger_version'] = logger_version

    # Count the number of user actions for the current group
    row_dict['total_actions'] = fu.get_total_actions(scene_df)

    # Append the row to the list
    rows_list.append(row_dict)

# Create a data frame from the list of rows
total_actions_df = DataFrame(rows_list)

# Group by session UUID and sum total actions
df = total_actions_df.groupby('session_uuid').sum()

# Create a mask for sessions with zero total actions
mask_series = (df.total_actions == 0)

# Get a list of session UUIDs with no user actions
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter for UUIDs with no user actions
mask_series = frvrs_logs_df.session_uuid.isin(session_uuids_list)

# Get unique file names associated with sessions with zero total actions
files_list = frvrs_logs_df[mask_series].file_name.unique().tolist()

# Check if there are files with zero actions
if files_list:
    
    # Create a formatted string of anomalous files
    print_str = f'\n\nThese files have no user action taken:\n\t{nu.conjunctify_nouns(files_list)}'
    
    # Add carriage returns for readability
    print_str = print_str.replace(' and ', ', and\n\t')

    # Print and update the anomalous files string and set
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These files have no user action taken:
	Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv, and
	7f9fbaf9-4b2a-42c6-a5f9-c5bb972ca11d.csv


In [7]:

# Initialize an empty list to store dictionaries for each session for Total Number of Teleports
rows_list = []

# Iterate through the scenes
for (session_uuid, scene_index), scene_df in fu.get_session_groupby(extra_column='scene_index'):
    
    # Create a dictionary to store information for each scene
    row_dict = {}
    
    # Get the logger version for the current session
    logger_version = fu.get_logger_version(scene_df)

    # Add the logger version, session_uuid, and scene_index to the row dictionary
    for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)
    row_dict['logger_version'] = logger_version

    # Get the number of teleports for the current scene and add it to the row dictionary
    row_dict['teleport_count'] = fu.get_teleport_count(scene_df)
    
    # Append the dictionary to the list
    rows_list.append(row_dict)

# Create a data frame from the list of dictionaries
teleport_count_df = DataFrame(rows_list)

# Group the data frame by UUID and sum the teleport counts
df = teleport_count_df.groupby('session_uuid').sum()

# Create a mask to filter sessions with teleport count equal to 0
mask_series = (df.teleport_count == 0)

# Get a list of session UUIDs with no teleportation
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter files associated with the identified session UUIDs
mask_series = frvrs_logs_df.session_uuid.isin(session_uuids_list)

# Get the file names with no teleportation being done
files_list = frvrs_logs_df[mask_series].file_name.unique().tolist()

# Check if there are files with no teleportation
if files_list:

    # Create a printable string of the file names
    print_str = f'\n\nThese files have no teleportation being done:\n\t{nu.conjunctify_nouns(files_list)}'
    
    # Format the printed string for readability
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')
    
    # Print the list of files and update the anomalous_files_str and set
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These files have no teleportation being done:
	Clean 2310f107-d9d2-418e-a2d7-dd7a17924544.csv,
	Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv,
	Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv,
	clean-2dd0d21c-1633-47de-848a-317df7ea3180.csv, and
	7f9fbaf9-4b2a-42c6-a5f9-c5bb972ca11d.csv


In [8]:

# Create a mask to filter rows where action_type is PULSE_TAKEN or INJURY_TREATED
mask_series = frvrs_logs_df.action_type.isin(['PULSE_TAKEN', 'INJURY_TREATED'])

# Define columns to group by and initialize lists


rows_list = []
indexes_list = []

# Iterate over groups based on the specified columns
for (session_uuid, scene_index, patient_id), patient_df in frvrs_logs_df[mask_series].groupby(fu.patient_groupby_columns):

    # Create a tuple to store the index of the current row
    index_tuple = (session_uuid, scene_index, patient_id)
    indexes_list.append(index_tuple)

    # Create a dictionary to store the results for the current row
    row_dict = {}

    # Add the logger_version for the current row
    row_dict['logger_version'] = fu.get_logger_version(patient_df)
    
    # Count the number of 'PULSE_TAKEN' actions in this group
    row_dict['pulses_count'] = fu.get_pulse_taken_count(patient_df)
    
    # Count the number of 'INJURY_TREATED' actions in this group
    row_dict['treated_count'] = fu.get_injury_treated_count(patient_df)
    
    # Try to calculate the number of pulses taken per injury treated
    try: row_dict['pulses_by_treated'] = row_dict['pulses_count'] / row_dict['treated_count']

    # Handle the case where 'treated_count' is zero to avoid division by zero
    except ZeroDivisionError: row_dict['pulses_by_treated'] = np.nan
    
    # Add the row_dict to the rows_list
    rows_list.append(row_dict)

# Create a data frame from the rows and set a multi index based on the grouped columns
pulses_count_df = DataFrame(rows_list, index=pd.MultiIndex.from_tuples(tuples=indexes_list, names=fu.patient_groupby_columns))

# Group the pulses count data frame by session_uuid and sum the values
df = pulses_count_df.groupby('session_uuid').sum()

# Check for sessions with 'treated_count' equal to zero
mask_series = (df.treated_count == 0)
session_uuids_list = df[mask_series].index.tolist()

# Filter logs for sessions with no 'INJURY_TREATED' actions
mask_series = frvrs_logs_df.session_uuid.isin(session_uuids_list)
files_list = frvrs_logs_df[mask_series].file_name.unique().tolist()

# Print files with no injury treatment being done
if files_list:
    print_str = f'\n\nThese files have no injury treatment being done:\n\t{nu.conjunctify_nouns(files_list)}'
    print_str = print_str.replace(' and ', ', and\n\t')
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)

# Check for sessions with 'pulses_count' equal to zero
mask_series = (df.pulses_count == 0)
session_uuids_list = df[mask_series].index.tolist()

# Filter logs for sessions with no 'PULSE_TAKEN' actions
mask_series = frvrs_logs_df.session_uuid.isin(session_uuids_list)
files_list = frvrs_logs_df[mask_series].file_name.unique().tolist()

# Print files with no pulses being taken
if files_list:
    print_str = f'\n\nThese files have no pulses being taken:\n\t{nu.conjunctify_nouns(files_list)}'
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These files have no injury treatment being done:
	Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv, and
	clean-2dd0d21c-1633-47de-848a-317df7ea3180.csv


These files have no pulses being taken:
	Clean 274ce308-c366-4829-b9cc-d535b51d16d3.csv,
	Clean 5fa79a8e-a2df-4bb9-b614-f3ce36a5edb0.csv,
	Clean a6b7ff70-3b20-48c6-86e8-744bad19f7d7.csv,
	Clean d90f2d85-e91c-4f12-b070-5929d95be1c5.csv,
	clean-446f0cb1-dceb-40f4-8bba-db5c07f67571.csv, and
	0b630a14-5acd-4e11-99df-8f59d804f807.csv


In [9]:

# Patient accuracy rate (how many patients correct / number of patients treated)

# Initialize an empty list to store dictionaries
rows_list = []

# Iterate over the scenes
for (session_uuid, scene_index), scene_df in fu.get_session_groupby(extra_column='scene_index'):

    # Get the logger version for the group
    logger_version = fu.get_logger_version(scene_df)
    
    # Create a dictionary to store the results for the group
    row_dict = {}
    for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)
    row_dict['logger_version'] = logger_version

    # Get the total number of patients treated
    total_treated = fu.get_injury_treated_count(scene_df)

    # If there were any patients treated, calculate the patient accuracy rate
    if total_treated:
        
        # Filter the dataframe to only include rows where the injury was treated correctly
        correctly_treated = fu.get_injury_correctly_treated_count(scene_df)

        # Calculate the patient accuracy rate
        row_dict['injury_treated_total_treated'] = total_treated
        row_dict['injury_treated_correctly_treated'] = correctly_treated
        row_dict['injury_treated_patient_accuracy_rate'] = correctly_treated / total_treated
    
    # Filter the data frame to only include rows where the injury record was treated.
    total_mask = (scene_df.injury_record_injury_treated == True)
    df2 = scene_df[total_mask]

    # Get the total number of patients treated
    total_treated = df2.shape[0]

    # If there were any patients treated, calculate the patient accuracy rate
    if total_treated:
        
        # Filter the dataframe to only include rows where the injury record was treated correctly
        correct_mask = (df2.injury_record_injury_treated_with_wrong_treatment == False)
        correctly_treated = df2[correct_mask].shape[0]

        # Calculate the patient accuracy rate
        row_dict['injury_record_total_treated'] = total_treated
        row_dict['injury_record_correctly_treated'] = correctly_treated
        row_dict['injury_record_patient_accuracy_rate'] = correctly_treated / total_treated

    # Add the row dictionary to the list of results
    rows_list.append(row_dict)

# Create a data frame from the list of dictionaries
patient_accuracy_rate_df = DataFrame(rows_list)

# Modalize into one patient accuracy rate column if possible
if 'injury_record_patient_accuracy_rate' in patient_accuracy_rate_df.columns:

    # Get a list of columns that contain the patient accuracy rate
    columns_list = [
        'injury_treated_patient_accuracy_rate', 'injury_record_patient_accuracy_rate'
    ]
    
    # Check if there's only one unique value across the specified columns
    mask_series = (patient_accuracy_rate_df[columns_list].apply(pd.Series.nunique, axis='columns') == 1)
    
    # Set the patient accuracy rate column for the rows identified by the mask series to the non-null value in one of the patient accuracy rate columns
    patient_accuracy_rate_df.loc[~mask_series, 'patient_accuracy_rate'] = np.nan
    
    # Define a function to select the first valid value
    def f(srs):
        cn = srs.first_valid_index()
        
        return srs[cn]
    
    # Modalize the patient accuracy rate columns to get a single column
    patient_accuracy_rate = patient_accuracy_rate_df[mask_series][columns_list].apply(f, axis='columns')
    
    # Set the patient accuracy rate column for the rows identified by the mask series to the single patient accuracy rate column
    patient_accuracy_rate_df.loc[mask_series, 'patient_accuracy_rate'] = patient_accuracy_rate
    
    # Group by 'session_uuid' and sum the patient accuracy rates
    df = patient_accuracy_rate_df.groupby('session_uuid').sum()
    mask_series = (df.patient_accuracy_rate == 0)
else:
    
    # Group by 'session_uuid' and sum the patient accuracy rates for 'injury_treated' data
    df = patient_accuracy_rate_df.groupby('session_uuid').sum()
    mask_series = (df.injury_treated_patient_accuracy_rate == 0)

# Get a list of session_uuids with no patient accuracy rate above zero
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter rows in 'frvrs_logs_df' based on session_uuids
mask_series = frvrs_logs_df.session_uuid.isin(session_uuids_list)

# Get a list of unique file names from the filtered rows
files_list = frvrs_logs_df[mask_series].file_name.unique().tolist()

# Print the list of files with no patient accuracy rate above zero
if files_list:
    print_str = f'\n\nThese files have no patient accuracy rate above zero:\n\t{nu.conjunctify_nouns(files_list)}'
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')
    print(print_str); anomalous_files_str += print_str
    anomalous_files_set.update(files_list)



These files have no patient accuracy rate above zero:
	Clean 158e6365-673b-4030-8b36-6704be5996a2.csv,
	Clean 2310f107-d9d2-418e-a2d7-dd7a17924544.csv,
	Clean 845d87c5-7b8b-4bf3-bfc6-91c74e285243.csv,
	Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv,
	Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv,
	clean-2dd0d21c-1633-47de-848a-317df7ea3180.csv, and
	7f9fbaf9-4b2a-42c6-a5f9-c5bb972ca11d.csv


In [10]:

# Number of patients engaged

# Initialize an empty list to store rows
rows_list = []

# Iterate over the sessions, grouped by scene
for (session_uuid, scene_index), scene_df in fu.get_session_groupby(extra_column='scene_index'):

    # Get the logger version
    logger_version = fu.get_logger_version(scene_df)

    # Get the number of patients in the session
    patients_count = fu.get_patient_count(scene_df)

    # Initialize the number of patients engaged
    patients_engaged = 0
    
    # Loop over patients within this scene
    for patient_id, patient_df in scene_df.groupby('patient_id'):
        
        # Create a mask to filter rows where action_type is 'PATIENT_ENGAGED'
        mask_series = (patient_df.action_type == 'PATIENT_ENGAGED')
        
        # If there are any rows that match the mask, increment the number of patients engaged
        if patient_df[mask_series].shape[0]: patients_engaged += 1

    # Check if there are patients in this scene
    if patients_count:
        # Create a dictionary to store row data
        row_dict = {}

        # Add the session UUID and the scene to the dictionary
        for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)

        # Add the logger version to the dictionary
        row_dict['logger_version'] = logger_version

        # Add the number of patients to the dictionary
        row_dict['patients_count'] = patients_count

        # Add the number of patients engaged to the dictionary
        row_dict['patients_engaged'] = patients_engaged

        # Calculate the percentage of patients engaged
        row_dict['percentage_engaged'] = patients_engaged / patients_count

        # Add the row dictionary to the list of rows
        rows_list.append(row_dict)

# Create a data frame from the list of rows
percentage_engaged_df = DataFrame(rows_list)

# Group the DataFrame by UUID and sum the number of patients engaged
df = percentage_engaged_df.groupby('session_uuid').sum()

# Create a mask to identify UUIDs with no engaged patients
mask_series = (df.patients_engaged == 0)

# List the UUIDs that meet the condition
session_uuids_list = df[mask_series].index.tolist()

# Create a mask to filter rows in the main data frame based on UUID
mask_series = frvrs_logs_df.session_uuid.isin(session_uuids_list)

# Get unique file_names that match the UUIDs
files_list = frvrs_logs_df[mask_series].file_name.unique().tolist()

# Check if there are any files with no patients engaged
if files_list:

    # Create a formatted string to list files with no engaged patients
    print_str = f'\n\nThese files have no patients being engaged:\n\t{nu.conjunctify_nouns(files_list)}'
    
    # Format the string to have line breaks after commas
    print_str = print_str.replace(', ', ',\n\t').replace(',\n\tand ', ', and\n\t')

    # Print the results and add the results to the anomalous_files_str string
    print(print_str); anomalous_files_str += print_str

    # Add the files to the anomalous_files_set set
    anomalous_files_set.update(files_list)



These files have no patients being engaged:
	Clean 158e6365-673b-4030-8b36-6704be5996a2.csv,
	Clean 2310f107-d9d2-418e-a2d7-dd7a17924544.csv,
	Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv,
	Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv,
	clean-2dd0d21c-1633-47de-848a-317df7ea3180.csv,
	clean-446f0cb1-dceb-40f4-8bba-db5c07f67571.csv, and
	7f9fbaf9-4b2a-42c6-a5f9-c5bb972ca11d.csv


In [11]:

print("\nDoug,\n\nHere is the set of anomalous files I'm concerned about:")
print('\t' + '\n\t'.join(sorted(anomalous_files_set)))
print(anomalous_files_str)


Doug,

Here is the set of anomalous files I'm concerned about:
	0b630a14-5acd-4e11-99df-8f59d804f807.csv
	7f9fbaf9-4b2a-42c6-a5f9-c5bb972ca11d.csv
	Clean 158e6365-673b-4030-8b36-6704be5996a2.csv
	Clean 2310f107-d9d2-418e-a2d7-dd7a17924544.csv
	Clean 274ce308-c366-4829-b9cc-d535b51d16d3.csv
	Clean 5fa79a8e-a2df-4bb9-b614-f3ce36a5edb0.csv
	Clean 845d87c5-7b8b-4bf3-bfc6-91c74e285243.csv
	Clean a6b7ff70-3b20-48c6-86e8-744bad19f7d7.csv
	Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv
	Clean d90f2d85-e91c-4f12-b070-5929d95be1c5.csv
	Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv
	clean-2dd0d21c-1633-47de-848a-317df7ea3180.csv
	clean-446f0cb1-dceb-40f4-8bba-db5c07f67571.csv


These files have no user action taken:
	Clean e78faf41-7bbd-410b-8750-e4e72b951216.csv, and
	7f9fbaf9-4b2a-42c6-a5f9-c5bb972ca11d.csv

These files have no teleportation being done:
	Clean 2310f107-d9d2-418e-a2d7-dd7a17924544.csv,
	Clean c6a48228-d864-4b20-93dd-8ad0d78d59c0.csv,
	Clean e78faf41-7bbd-410b-8750-e4e72b951216